In [1]:
## Query real time data from DingXiangYuan, and keep the latest records every day for each city

## Created on Sat Feb  8 12:41:50 2020
#### resource: https://github.com/jianxu305/nCov2019_analysis

import pandas as pd
import matplotlib.pyplot as plt
import utils   # some convenient functions
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
DXYArea = utils.load_chinese_data() # Query latest Regional Data from DXY
DXYArea.head(2)  

Latest Update Time:  2020-02-08 19:25:32.088000
Data Range:  2020-01-24 to 2020-02-08
Data Size:  25827


,provinceName,cityName,province_confirmedCount,province_suspectedCount,province_curedCount,province_deadCount,city_confirmedCount,city_suspectedCount,city_curedCount,city_deadCount,updateTime,updateDate
0,山东省,青岛,416,0,44,0,46,0,7,0,2020-02-08 19:25:32.088,2020-02-08
1,山东省,济南,416,0,44,0,42,0,3,0,2020-02-08 19:25:32.088,2020-02-08


In [3]:
daily_frm_DXYArea = utils.aggDaily(DXYArea)  # aggregate to daily data (keep the latest records every day)

In [4]:
len(DXYArea)

25827

In [5]:
len(daily_frm_DXYArea)

5684

In [6]:
daily_frm_DXYArea.to_csv ('../data/DXYArea.csv', index = None, header=True)